In [ ]:
google_colab = True
if google_colab:
  %tensorflow_version 2.x

In [ ]:
if google_colab:
  from google.colab import drive
  drive.mount('/content/drive')

In [ ]:
if google_colab:
  !unzip ann-and-dl-image-segmentation.zip

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"]="-1" 
import tensorflow as tf
import numpy as np

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 8435
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

# Set GPU memory growth
# Allows to only as much GPU memory as needed
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


# Image Segmentation

In [3]:
# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

# Create training ImageDataGenerator object
# We need two different generators for images and corresponding masks
if apply_data_augmentation:
    train_img_data_gen = ImageDataGenerator(rotation_range=10,
                                            width_shift_range=10,
                                            height_shift_range=10,
                                            zoom_range=0.3,
                                            horizontal_flip=True,
                                            vertical_flip=True,
                                            fill_mode='constant',
                                            cval=0,
                                            validation_split=0.2,
                                            rescale=1./255,)
    train_mask_data_gen = ImageDataGenerator(rotation_range=10,
                                             width_shift_range=10,
                                             height_shift_range=10,
                                             zoom_range=0.3,
                                             horizontal_flip=True,
                                             vertical_flip=True,
                                             fill_mode='constant',
                                             cval=0,
                                             validation_split=0.2,
                                             rescale=1./255)
else:
    train_img_data_gen = ImageDataGenerator(rescale=1./255,
                                            validation_split=0.2)
    train_mask_data_gen = ImageDataGenerator(rescale=1./255,
                                             validation_split=0.2)

In [4]:
# Create generators to read images from dataset directory
# -------------------------------------------------------
dataset_dir = os.path.join(cwd, 'Segmentation_Dataset')

# Batch size
bs = 16

# img shape
img_h = 256
img_w = 256

num_classes=2

training_dir = os.path.join(dataset_dir, 'training')
# Training
train_img_gen = train_img_data_gen.flow_from_directory(os.path.join(training_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None,
                                                       shuffle=True,
                                                       subset='training',
                                                       interpolation='bilinear',
                                                       color_mode='rgb',
                                                       seed=SEED)  
train_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs,
                                                         class_mode=None,
                                                         shuffle=True,
                                                         subset='training',
                                                         interpolation='bilinear',
                                                         color_mode='grayscale',
                                                         seed=SEED)
train_gen = zip(train_img_gen, train_mask_gen)

# Validation
valid_img_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None,
                                                       shuffle=False,
                                                       subset='validation',
                                                       interpolation='bilinear',
                                                       color_mode='rgb',
                                                       seed=SEED)
valid_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs, 
                                                         class_mode=None,
                                                         shuffle=False,
                                                         subset='validation',
                                                         interpolation='bilinear',
                                                         color_mode='grayscale',
                                                         seed=SEED)
valid_gen = zip(valid_img_gen, valid_mask_gen)

Found 6118 images belonging to 1 classes.
Found 6118 images belonging to 1 classes.
Found 1529 images belonging to 1 classes.
Found 1529 images belonging to 1 classes.


In [5]:
# Create Dataset objects
# ----------------------

# Training
# --------
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 1]))

def prepare_target(x_, y_):
    y_ = tf.cast(tf.expand_dims(y_[..., 0], -1), tf.int32)
    return x_, tf.where(y_ > 0, y_ - 1, y_ + 1)

train_dataset = train_dataset.map(prepare_target)

# Repeat
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 1]))
valid_dataset = valid_dataset.map(prepare_target)

# Repeat
valid_dataset = valid_dataset.repeat()

## Convolutional Neural Network (CNN)
### Encoder-Decoder

In [6]:
# Create Model
# ------------

def create_model(depth, start_f, num_classes, dynamic_input_shape):

    model = tf.keras.Sequential()
    
    # Encoder
    # -------
    for i in range(depth):
        
        if i == 0:
            if dynamic_input_shape:
                input_shape = [None, None, 3]
            else:
                input_shape = [img_h, img_w, 3]
        else:
            input_shape=[None]
        
        model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same',
                                         input_shape=input_shape))
        model.add(tf.keras.layers.ReLU())
        model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

        start_f *= 2

    # Decoder
    # -------
    for i in range(depth):
        model.add(tf.keras.layers.UpSampling2D(2, interpolation='bilinear'))
        model.add(tf.keras.layers.Conv2D(filters=start_f // 2,
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same'))

        model.add(tf.keras.layers.ReLU())

        start_f = start_f // 2

    # Prediction Layer
    # ----------------
    model.add(tf.keras.layers.Conv2D(filters=num_classes,
                                     kernel_size=(1, 1),
                                     strides=(1, 1),
                                     padding='same',
                                     activation='sigmoid'))
    
    return model

In [7]:
model = create_model(depth=6, 
                     start_f=4, 
                     num_classes=2, 
                     dynamic_input_shape=False)

# Visualize created model as a table
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 4)       112       
_________________________________________________________________
re_lu (ReLU)                 (None, 256, 256, 4)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 4)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 8)       296       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 128, 128, 8)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 16)        1

## Prepare the model for training

In [8]:
# Optimization params
# -------------------

# Loss
# Sparse Categorical Crossentropy to use integers (mask) instead of one-hot encoded labels
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False) 
# learning rate
lr = 5e-2
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------
def my_IoU(y_true, y_pred):
    # from probability to predicted class {0, 1}
    y_pred = tf.cast(y_pred > 0.5, tf.float32) # when using sigmoid. Use argmax for softmax

    # A and B
    intersection = tf.reduce_sum(y_true * y_pred)
    # A or B
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    # IoU
    return intersection / union

metrics = [['accuracy', my_IoU]]
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## Training with callbacks

In [9]:
import os
from datetime import datetime

cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'segmentation_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=0)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)


model.fit(x=train_dataset,
          epochs=70,
          steps_per_epoch=len(train_img_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_img_gen), 
          callbacks=callbacks)

Train for 383 steps, validate for 96 steps
Epoch 1/70
383/383 [==============================] - 193s 503ms/step - loss: 0.5452 - accuracy: 0.7427 - my_IoU: 0.7929 - val_loss: 0.4922 - val_accuracy: 0.7455 - val_my_IoU: 0.7458
Epoch 2/70
383/383 [==============================] - 120s 313ms/step - loss: 0.4843 - accuracy: 0.7432 - my_IoU: 0.7418 - val_loss: 0.4668 - val_accuracy: 0.7438 - val_my_IoU: 0.7126
Epoch 3/70
383/383 [==============================] - 120s 313ms/step - loss: 0.4613 - accuracy: 0.7694 - my_IoU: 0.7420 - val_loss: 0.4461 - val_accuracy: 0.7916 - val_my_IoU: 0.7440
Epoch 4/70
383/383 [==============================] - 118s 307ms/step - loss: 0.4393 - accuracy: 0.7863 - my_IoU: 0.7473 - val_loss: 0.4269 - val_accuracy: 0.7931 - val_my_IoU: 0.7523
Epoch 5/70
383/383 [==============================] - 117s 306ms/step - loss: 0.4187 - accuracy: 0.8002 - my_IoU: 0.7470 - val_loss: 0.4102 - val_accuracy: 0.8088 - val_my_IoU: 0.7437
Epoch 6/70
383/383 [=================

## Compute prediction

In [10]:
from datetime import datetime

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(csv_fname, 'w') as f:

      f.write('ImageId,EncodedPixels,Width,Height\n')

      for key, value in results.items():
          f.write(key + ',' + str(value) + ',' + '256' + ',' + '256' + '\n')

In [11]:
def rle_encode(img):
      # Flatten column-wise
      pixels = img.T.flatten()
      pixels = np.concatenate([[0], pixels, [0]])
      runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
      runs[1::2] -= runs[::2]
      return ' '.join(str(x) for x in runs)

In [12]:
import time
import PIL

from PIL import Image

# Cycle over test images

test_dir = os.path.join(dataset_dir, 'test')
test_img_dir = os.path.join(test_dir, 'images', 'img')

img_filenames = next(os.walk(test_img_dir))[2]

results={}

for img_filename in img_filenames:
    
    img = Image.open(os.path.join(test_img_dir, img_filename)).convert('RGB')
    
    img_filename = img_filename[:-4]
    img_arr = np.expand_dims(np.array(img), 0)
    img_arr = img_arr / 255.0
    out_softmax = model.predict(x=img_arr)

    # Get predicted class as the index corresponding to the maximum value in the vector probability
    predicted_class = tf.argmax(out_softmax, -1)
    predicted_class = predicted_class[0]
    prediction_img = np.zeros([256, 256, 1])
    prediction_img[np.where(predicted_class == 0)] = 1
    prediction_img[np.where(predicted_class == 1)] = 0
    results[img_filename] = rle_encode(prediction_img)

create_csv(results)